# The Genesis process

Creating a new game requires the creation of a system and planets, factions and people.

This notebook tests that process as well as documents it. 

In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

%load_ext lab_black

In [2]:
from app import creators
from app import objects

In [3]:
from app.creators import homeworld
from app.creators import universe

An example `data` set for testing. This is the user form that the user submits when creating a new game. 

You can get this from the template at `app\templates\app\creation\genesis.js`


In [4]:
data = {
    "label": "form",
    "name": "worldgenform",
    "objid": "0000000000001",
    "owner": "user.username",
    "username": "user.username",
    "accountid": "0000000000001",
    "conformity": 0.5,
    "constitution": 0.5,
    "literacy": 0.5,
    "aggression": 0.5,
    "num_planets": 4,
    "num_moons": 10,
    "starting_pop": 7,
    "organics": 0.5,
    "minerals": 0.5,
}

# The Solar System

In [5]:
homesystem_data = universe.build_homeSystem(data, username="notebook")
homesystem_data

([{'label': 'form',
   'name': 'worldgenform',
   'objid': '0000000000001',
   'owner': 'user.username',
   'username': 'user.username',
   'accountid': '0000000000001',
   'conformity': 0.5,
   'constitution': 0.5,
   'literacy': 0.5,
   'aggression': 0.5,
   'num_planets': 4,
   'num_moons': 10,
   'starting_pop': 7,
   'organics': 0.5,
   'minerals': 0.5},
  {'name': 'Hat',
   'label': 'system',
   'isHomeSystem': 'true',
   'objid': '7717709550411',
   'glat': 6.617,
   'glon': -31.803,
   'gelat': -4.362},
  {'name': 'Vel',
   'class': 'G',
   'objid': '2821271265988',
   'label': 'star',
   'radius': 106},
  {'name': 'Khaburgsta',
   'class': 'terrestrial',
   'objid': '5659344590793',
   'label': 'moon',
   'orbitsId': '7898698359258',
   'orbitsName': 'Zulas',
   'orbitsDistance': 1.274,
   'mass': 0.015331756733681357,
   'radius': 0.0511306129510325,
   'isSupportsLife': False,
   'isPopulated': False},
  {'name': 'Bingaltro',
   'class': 'rocky',
   'objid': '3783711313991',

# The planet Surface

In [6]:
homeworld_data = homeworld.build_people(data)
homeworld_data

([{'name': 'Badla',
   'objid': '2152333961637',
   'label': 'species',
   'consumes': ['Organic'],
   'effuses': ['Organic waste', 'Plastics'],
   'viral_resilience': 0.7,
   'habitat_resilience': 0.2},
  {'name': 'Fucas Hyjiangfuca',
   'objid': '6714538532858',
   'label': 'pop',
   'conformity': 0.574,
   'literacy': 0.277,
   'aggression': 0.487,
   'constitution': 0.577,
   'health': 0.5,
   'isInFaction': '8424616458051',
   'industry': 0.532,
   'wealth': 0.4045,
   'factionLoyalty': 0.655,
   'isIdle': 'true'},
  {'name': 'Rosar Orbarnu',
   'objid': '8368216969422',
   'label': 'pop',
   'conformity': 0.606,
   'literacy': 0.605,
   'aggression': 0.553,
   'constitution': 0.547,
   'health': 0.5,
   'isInFaction': '8422192744305',
   'industry': 0.55,
   'wealth': 0.5775,
   'factionLoyalty': 0.778,
   'isIdle': 'true'},
  {'name': 'Rosar Cor',
   'objid': '2956170443012',
   'label': 'pop',
   'conformity': 0.604,
   'literacy': 0.526,
   'aggression': 0.597,
   'constitutio